In [1]:
#!pip install translators
#import translators as ts
#ts.translate_text('mansions', translator = 'google', to_language = 'ru')

#C:\Users\Evgeny Larin\anaconda_projects\MoMValkyrieTranslations\Possessed.valkyrie

In [2]:
#функция для удаления файла из zip архива https://stackoverflow.com/questions/4653768/overwriting-file-in-ziparchive
import tempfile
import zipfile
import shutil
import os

def remove_from_zip(zipfname, *filenames):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with zipfile.ZipFile(zipfname, 'r') as zipread:
            with zipfile.ZipFile(tempname, 'w') as zipwrite:
                for item in zipread.infolist():
                    if item.filename not in filenames:
                        data = zipread.read(item.filename)
                        zipwrite.writestr(item, data)
        shutil.move(tempname, zipfname)
    finally:
        shutil.rmtree(tempdir)

In [3]:
from zipfile import ZipFile
import pandas as pd
import os

# Укажите путь до файла Localization.English.txt

In [4]:
path = input('Укажите путь к файлу сюжета .valkyrie\n')
z = ZipFile(fr'{path}', 'a')

files_list = [text_file.filename for text_file in z.infolist() ]

if 'Localization.Russian.txt' in files_list:
    print('Сюжет уже имеет файл с переводом Localization.Russian.txt')
else:
    print(0)

Укажите путь к файлу сюжета .valkyrie
C:\Users\Evgeny Larin\anaconda_projects\MoMValkyrieTranslations\Possessed.valkyrie
0


In [5]:
with z.open('Localization.English.txt') as f:
    df = pd.read_csv(f,
                     sep='^([^,]+),',
                     engine='python',
                     header = None,
                     usecols = range(1,3))
df.rename(columns={2:'text'}, inplace = True)

In [6]:
#ручные замены
df = df.replace(to_replace = '</i\|\|\|', value = '</i>|||', regex = True)

In [7]:
special = df.text[df.text.str.contains('{') == True].unique().tolist() #ищем все строчки, в которых есть {}
sep = '}'
list2 = []
for i in special:  #отделяем {} от {} если они в одной строчке
    if i.find('{') >= 0 and i.find('}') >= 0:
        list2.extend([x+sep for x in i.split(sep)])

special = []       #чистим мусор (наверное можно оптимизировать, пока хз как)
for i in list2:
    start = i.find('{')
    if start >= 0:
        end = i.find('}')+1
        special.append(i[start:end])
        
special = set(special) #удаляем дубликаты

In [8]:
special2 = df.text[df.text.str.contains('<') == True].unique() #ищем все строчки, в которых есть <>
sep = '>'
list2 = []
for i in special2:  #отделяем {} от {} если они в одной строчке
    if i.find('<') >= 0 and i.find('>') >= 0:
        list2.extend([x+sep for x in i.split(sep)])
        
special2 = []       #чистим мусор (наверное можно оптимизировать, пока хз как)
for i in list2:
    start = i.find('<')
    if start >= 0:
        end = i.find('>')+1
        special2.append(i[start:end])
special2 = set(special2) #удаляем дубликаты

In [9]:
special_list = []
special_dict = {}
n = 0
for i in special: #собираем все специальные значения
    start = i.find('{')
    end = i.find('}', start)+1
    special_list.append(i[start:end])
special_list = set(special_list) #сохраняем только уникальные специальные значения
for i in special_list: #сохраняем уникальные значения в словарь и присваиваем порядковые номера
    n = n+1
    special_dict[i] = '<'+str(n)+'>'

In [10]:
special_list2 = []
special_dict2 = {}
n = 0
for i in special2: #собираем все специальные значения
    start = i.find('<')
    end = i.find('>', start)+1
    special_list2.append(i[start:end])
special_list2 = set(special_list2) #сохраняем только уникальные специальные значения
for i in special_list2: #сохраняем уникальные значения в словарь и присваиваем порядковые номера
    n = n-1
    special_dict2[i] = '<'+str(n)+'>' 

In [11]:
#заменяем специальные значения на порядковые номера (чтобы переводчик их не переводил) 
df["text"] = df.text.replace(special_dict2, regex=True).replace(special_dict, regex=True)
df.head(11)

,1,text
0,.,English
1,quest.name,Possessed 1.2
2,quest.description,"""This horror story plays in a remote, abandone..."
3,quest.synopsys,To correct the horrors from your families' his...
4,quest.authors,"""Story: <-15>Anton Timmermans & William van de..."
5,ATTACK,Attack
6,CANCEL,Cancel
7,CONTINUE,Continue
8,GUESS,Guess
9,LOOK,Look


In [12]:
#сохраняем текст для перевода
name = 'TextForTranslate.xlsx'
df.text.to_excel(f'{name}')

# Теперь нужно перевесити полученный файл в Google Переводчике

# https://translate.google.com/?hl=ru&tab=TT&sl=en&tl=ru&op=docs

In [13]:
path2 = input('Путь к переведенному файлу\n')

Путь к переведенному файлу
C:\Users\Evgeny Larin\Downloads


In [14]:
#считываем переведенный файл
df2 = pd.read_excel(fr'{path2}\\{name}', usecols = [1]) 

In [15]:
#иногда переводчик добавляет лишние пробелы или неправильно распознает специальные символы - исправляем
df2 = df2.replace(to_replace = ' >', value = '>', regex = True)\
         .replace(to_replace = '\\\п', value = '\\n', regex = True)

In [16]:
#слепляем столбец триггеров и столбец с переведенным текстом
df_done = df.merge(df2, left_index=True, right_index=True).drop(columns=['text'])
df_done.rename(columns = {'текст':'text'}, inplace = True)

In [17]:
#создаем инвертированный словарь и заменяем порядковые номера на специальные значения
dict_special = dict(zip(special_dict.values(), special_dict.keys()))
dict_special2 = dict(zip(special_dict2.values(), special_dict2.keys()))
df_done['text'] = df_done['text'].replace(dict_special, regex=True).replace(dict_special2, regex=True).replace('Английский', 'Russian', regex = True)

In [18]:
#удаляем пробелы, который добавляет переводчик 
df_done['text'] =  df_done['text'].str.strip()

In [19]:
#добавляем файл в архив с сюжетом
df_done.to_csv('Localization.Russian.txt', sep = ',', index = False, header = False)
z.write('Localization.Russian.txt')
os.remove('Localization.Russian.txt')

In [20]:
#извлекаем файл quest.ini и удаляем его в архиве сюжета
z.extract('quest.ini')
z.close()
remove_from_zip(fr'{path}', 'quest.ini')    

In [21]:
#редактируем файл quest.ini и добавляем в архив с сюжетом
file = open('quest.ini', 'r')
pozition = file.readlines().index('[QuestText]\n')+1
file = open('quest.ini', 'r')
new_file = file.readlines()
elem = 'Localization.Russian.txt\n'
new_file.insert(pozition, elem)

with open('quest.ini', 'w') as file:
    for row in new_file:
        s = "".join(map(str, row))
        file.write(s)
        
z = ZipFile(fr'{path}', 'a')
z.write('quest.ini')
z.close()
os.remove('quest.ini')